# One-time pad
## Classic Crypto (FALL 2022-2023)

<h2>3.  One-time pad</h2>
Create a Python program that encrypts/decrypts messages using One-time pad.

<h3>3.1: Encryption & Decryption</h3>

For the encryption and decryption functions one must take into account the following:
* The one-time pad key should be randomly generated or provided by the user (justify your randomness choices)
* The plaintext/ciphertexts should be provided as an input text file
* All ASCII characters should be supported
* No substitution for non ASCII characters
* Error control
    * The one-time pad key must use only ASCII characters
    * Βinary files are not supported

<h3>3.2: Cryptanalysis</h3>
Create a function which, given a collection of ASCII text files that have been encrypted with the
same one-time pad key, should be able to decrypt as accurately as possible a target ciphertext
that has been encrypted with the same one-time pad key. You can assume that the length of the
content of all the encrypted ciphertexts is the same. Try automating this:
1. Iterate over the ciphertexts while XOR-ing each of them with the others
2. Identify the places where a space character has been XOR-ed with a letter character
3. If a space character has been found in an index more than 75% percent of the time, we
assume that index as a possible space character in the ciphertext that is being tested
Since space characters XOR-ed with a letter character simply change the case of the letter, by
XOR-ing all the texts with each other one can guess most of the characters in the target text.
While trying to decrypt the target ciphertext all still to be found characters should be replaced
with the asterisk symbol (*)

import string
import sys
import argparse
import secrets

WARNING: The code bellow is in comments because it needs file input to run which is not available with jupyter-book




    def otp_xor(plain: bytes, key: bytes):
        res = b""

        assert (len(plain) <= len(key))

        for p, k in zip(plain, key):
            if p < 128:
                res += bytes([p ^ k])
            else:
                res += bytes([p])

        return res


    def encrypt(plain: bytes, key: bytes):
        # Wrapper just for the naming
        return otp_xor(plain, key)


    def decrypt(plain: bytes, key: bytes):
        # Wrapper just for the naming
        return otp_xor(plain, key)


    def cryptanalysis(ciphertexts: list[bytes], target: bytes):
        key = bytearray([c ^ ord('*') for c in target])

        for cipher1 in ciphertexts:
            count = {}
            for cipher2 in ciphertexts:
                if not (cipher1 is cipher2):
                    for x_i, chars in enumerate(zip(cipher1, cipher2)):
                        if chr(chars[0] ^ chars[1]) in string.ascii_letters:
                            count[x_i] = count.get(x_i, 0)+1

            for x_i, c in count.items():
                if c/len(ciphertexts) >= 0.75:
                    key[x_i] = cipher1[x_i] ^ ord(' ')
    
        return bytes([t ^ k for t, k in zip(target, key)])

        # Perhaps use a dictionary to improve the results of the attack


    def main(arguments):

        parser = argparse.ArgumentParser(
            formatter_class=argparse.RawDescriptionHelpFormatter)
        parser.add_argument("-e", "--encrypt",
                        help="Encrypt text file", type=argparse.FileType('rb'))
        parser.add_argument(
            "-d", "--decrypt", help="Decrypt ciphertext file", type=argparse.FileType('rb'))
        parser.add_argument(
            "-p", "--otp", help="One-time pad used for encryption/decryption", type=argparse.FileType('rb'))
        parser.add_argument("-c", "--cryptanalysis", nargs="+",
                            help="Perform cryptanalysis on the given ciphertexts", type=argparse.FileType('rb'))
        parser.add_argument(
            "-t", "--target", help="Ciphertext to decrypt using the results from cryptanalysis of the given ciphertexts", type=argparse.FileType('rb'))

        parser.add_argument(
            "-o", "--output", help="The file where the result of the encryption/decryption will be written", type=argparse.FileType('wb'))

        args = parser.parse_args(arguments)

        if args.encrypt:
            file_content: bytes = args.encrypt.read()
            if args.otp:
                otp: bytes = args.otp.read()
                if len(file_content) > len(otp):
                    print("The key must be at least as long as the plaintext")
                    exit(1)
            else:
                otp = bytes([secrets.randbelow(128)
                             for _ in range(len(file_content))])
                print(
                    f"No key selected. Auto-generated key: {otp}", file=sys.stderr)
            # Replace this comment and the following statement with your code
            encrypted_text = encrypt(file_content, otp)
            if args.output:
                args.output.write(encrypted_text)
            else:
                print(
                    "WARNING: -o/--output is suggested with -e/--encrypt. Output to command line is unstable\n", file=sys.stderr)

                print(encrypted_text.decode())
            if args.decrypt:
            if args.otp:
                otp: bytes = args.otp.read()
                file_content: bytes = args.decrypt.read()
                # Replace this comment and the following statement with your code

                if len(file_content) > len(otp):
                    print("The key must be at least as long as the plaintext")
                    exit(1)

                decrypted_text = decrypt(file_content, otp)
                print(decrypted_text.decode())
            else:
                parser.error("-d/--decrypt requires -p/--otp")
        if args.cryptanalysis:
            if args.target:
                amount_of_ciphertexts = len(args.cryptanalysis)
                ciphertexts = []
                target_cipher_text = args.target.read()
                for i in range(amount_of_ciphertexts):
                    ciphertexts.append(args.cryptanalysis[i].read())
                # Replace this comment and the following statement with your code
                decrypted_text = cryptanalysis(ciphertexts, target_cipher_text)
                if args.output:
                    args.output.write(decrypted_text)
                else:
                    print(decrypted_text.decode())
            else:
                parser.error("-c/--cryptanalysis requires -t/--target")




    
    plaintext_path = "test_files/otp/target_plaintext.txt"

    sys.argv = ["",
            "-e", "test_files/otp/target_plaintext.txt",
            "-d", "test_files/otp/target.txt", 
            "-p","test_files/otp/otp.txt",
            "-c", "test_files/otp/c0.txt", 
            "-t", "test_files/otp/target.txt" 
            ]


    main(sys.argv[1:])
